In [2]:
import os
# Bertsum directory chdir
os.chdir('./src')

In [3]:
%ls

__pycache__/    models/  prepro/        req.txt           tmp/
distributed.py  others/  preprocess.py  requirements.txt  train.py


## 추출요약 코드

In [58]:
"""
    Main training workflow
"""
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time
import numpy as np

import torch
from pytorch_pretrained_bert import BertConfig


import distributed
from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.model_builder import Summarizer
from tensorboardX import SummaryWriter
from models.reporter import ReportMgr
from models.stats import Statistics
from others.logging import logger
# from models.trainer import build_trainer
# build_trainer의 dependency package pyrouge.utils가 import되지 않아 직접 셀에 삽입
from others.logging import logger, init_logger
import easydict

args = easydict.EasyDict({
    "encoder":'classifier',
    "mode":'summary',
    "bert_data_path":'../bert_sample/korean',
    "model_path":'../models/bert_classifier',
    "bert_model":'../001_bert_morp_pytorch',
    "result_path":'../results/korean',
    "temp_dir":'.',
    "bert_config_path":'../001_bert_morp_pytorch/bert_config.json',
    "batch_size":1000,
    "use_interval":True,
    "hidden_size":128,
    "ff_size":512,
    "heads":4,
    "inter_layers":2,
    "rnn_size":512,
    "param_init":0,
    "param_init_glorot":True,
    "dropout":0.1,
    "optim":'adam',
    "lr":2e-3,
    "report_every":1,
    "save_checkpoint_steps":5,
    "block_trigram":True,
    "recall_eval":False,
    
    "accum_count":1,
    "world_size":1,
    "visible_gpus":'-1',
    "gpu_ranks":'0',
    "log_file":'../logs/bert_classifier',
    "test_from":'../models/bert_classifier2/model_step_35000.pt'
})


def build_trainer(args, device_id, model,
                  optim):
    """
    Simplify `Trainer` creation based on user `opt`s*
    Args:
        opt (:obj:`Namespace`): user options (usually from argument parsing)
        model (:obj:`onmt.models.NMTModel`): the model to train
        fields (dict): dict of fields
        optim (:obj:`onmt.utils.Optimizer`): optimizer used during training
        data_type (str): string describing the type of data
            e.g. "text", "img", "audio"
        model_saver(:obj:`onmt.models.ModelSaverBase`): the utility object
            used to save the model
    """
    device = "cpu" if args.visible_gpus == '-1' else "cuda"


    grad_accum_count = args.accum_count
    n_gpu = args.world_size

    if device_id >= 0:
        gpu_rank = int(args.gpu_ranks[device_id])
    else:
        gpu_rank = 0
        n_gpu = 0

    #print('gpu_rank %d' % gpu_rank)

    tensorboard_log_dir = args.model_path

    writer = SummaryWriter(tensorboard_log_dir, comment="Unmt")

    report_manager = ReportMgr(args.report_every, start_time=-1, tensorboard_writer=writer)

    trainer = Trainer(args, model, optim, grad_accum_count, n_gpu, gpu_rank, report_manager)

    # print(tr)
    if (model):
        n_params = _tally_parameters(model)
        #logger.info('* number of parameters: %d' % n_params)

    return trainer

class Trainer(object):
    """
    Class that controls the training process.

    Args:
            model(:py:class:`onmt.models.model.NMTModel`): translation model
                to train
            train_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            valid_loss(:obj:`onmt.utils.loss.LossComputeBase`):
               training loss computation
            optim(:obj:`onmt.utils.optimizers.Optimizer`):
               the optimizer responsible for update
            trunc_size(int): length of truncated back propagation through time
            shard_size(int): compute loss in shards of this size for efficiency
            data_type(string): type of the source input: [text|img|audio]
            norm_method(string): normalization methods: [sents|tokens]
            grad_accum_count(int): accumulate gradients this many times.
            report_manager(:obj:`onmt.utils.ReportMgrBase`):
                the object that creates reports, or None
            model_saver(:obj:`onmt.models.ModelSaverBase`): the saver is
                used to save a checkpoint.
                Thus nothing will be saved if this parameter is None
    """

    def __init__(self,  args, model,  optim,
                  grad_accum_count=1, n_gpu=1, gpu_rank=1,
                  report_manager=None):
        # Basic attributes.
        self.args = args
        self.save_checkpoint_steps = args.save_checkpoint_steps
        self.model = model
        self.optim = optim
        self.grad_accum_count = grad_accum_count
        self.n_gpu = n_gpu
        self.gpu_rank = gpu_rank
        self.report_manager = report_manager

        self.loss = torch.nn.BCELoss(reduction='none')
        assert grad_accum_count > 0
        # Set model in training mode.
        if (model):
            self.model.train()

    def summary(self, test_iter, step, cal_lead=False, cal_oracle=False):
        """ Validate model.
            valid_iter: validate data iterator
        Returns:
            :obj:`nmt.Statistics`: validation loss statistics
        """
        # Set model in validating mode.
        def _get_ngrams(n, text):
            ngram_set = set()
            text_length = len(text)
            max_index_ngram_start = text_length - n
            for i in range(max_index_ngram_start + 1):
                ngram_set.add(tuple(text[i:i + n]))
            return ngram_set

        def _block_tri(c, p):
            tri_c = _get_ngrams(3, c.split())
            for s in p:
                tri_s = _get_ngrams(3, s.split())
                if len(tri_c.intersection(tri_s))>0:
                    return True
            return False

        if (not cal_lead and not cal_oracle):
            self.model.eval()
        stats = Statistics()

        
        with torch.no_grad():
            for batch in test_iter:
                src = batch.src
                labels = batch.labels
                segs = batch.segs
                clss = batch.clss
                mask = batch.mask
                mask_cls = batch.mask_cls


                gold = []
                pred = []

                if (cal_lead):
                    selected_ids = [list(range(batch.clss.size(1)))] * batch.batch_size
                elif (cal_oracle):
                    selected_ids = [[j for j in range(batch.clss.size(1)) if labels[i][j] == 1] for i in
                                    range(batch.batch_size)]
                else:
                    sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

                    # loss = self.loss(sent_scores, labels.float())
                    # loss = (loss * mask.float()).sum()
                    # batch_stats = Statistics(float(loss.cpu().data.numpy()), len(labels))
                    # stats.update(batch_stats)

                    sent_scores = sent_scores + mask.float()
                    sent_scores = sent_scores.cpu().data.numpy()
                    selected_ids = np.argsort(-sent_scores, 1)
                # selected_ids = np.sort(selected_ids,1)
                

        return selected_ids


    def _gradient_accumulation(self, true_batchs, normalization, total_stats,
                               report_stats):
        if self.grad_accum_count > 1:
            self.model.zero_grad()

        for batch in true_batchs:
            if self.grad_accum_count == 1:
                self.model.zero_grad()

            src = batch.src
            labels = batch.labels
            segs = batch.segs
            clss = batch.clss
            mask = batch.mask
            mask_cls = batch.mask_cls

            sent_scores, mask = self.model(src, segs, clss, mask, mask_cls)

            loss = self.loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()
            (loss/loss.numel()).backward()
            # loss.div(float(normalization)).backward()

            batch_stats = Statistics(float(loss.cpu().data.numpy()), normalization)


            total_stats.update(batch_stats)
            report_stats.update(batch_stats)

            # 4. Update the parameters and statistics.
            if self.grad_accum_count == 1:
                # Multi GPU gradient gather
                if self.n_gpu > 1:
                    grads = [p.grad.data for p in self.model.parameters()
                             if p.requires_grad
                             and p.grad is not None]
                    distributed.all_reduce_and_rescale_tensors(
                        grads, float(1))
                self.optim.step()

        # in case of multi step gradient accumulation,
        # update only after accum batches
        if self.grad_accum_count > 1:
            if self.n_gpu > 1:
                grads = [p.grad.data for p in self.model.parameters()
                         if p.requires_grad
                         and p.grad is not None]
                distributed.all_reduce_and_rescale_tensors(
                    grads, float(1))
            self.optim.step()

    def _save(self, step):
        real_model = self.model
        # real_generator = (self.generator.module
        #                   if isinstance(self.generator, torch.nn.DataParallel)
        #                   else self.generator)

        model_state_dict = real_model.state_dict()
        # generator_state_dict = real_generator.state_dict()
        checkpoint = {
            'model': model_state_dict,
            # 'generator': generator_state_dict,
            'opt': self.args,
            'optim': self.optim,
        }
        checkpoint_path = os.path.join(self.args.model_path, 'model_step_%d.pt' % step)
        #logger.info("Saving checkpoint %s" % checkpoint_path)
        # checkpoint_path = '%s_step_%d.pt' % (FLAGS.model_path, step)
        if (not os.path.exists(checkpoint_path)):
            torch.save(checkpoint, checkpoint_path)
            return checkpoint, checkpoint_path

    def _start_report_manager(self, start_time=None):
        """
        Simple function to start report manager (if any)
        """
        if self.report_manager is not None:
            if start_time is None:
                self.report_manager.start()
            else:
                self.report_manager.start_time = start_time

    def _maybe_gather_stats(self, stat):
        """
        Gather statistics in multi-processes cases

        Args:
            stat(:obj:onmt.utils.Statistics): a Statistics object to gather
                or None (it returns None in this case)

        Returns:
            stat: the updated (or unchanged) stat object
        """
        if stat is not None and self.n_gpu > 1:
            return Statistics.all_gather_stats(stat)
        return stat

    def _maybe_report_training(self, step, num_steps, learning_rate,
                               report_stats):
        """
        Simple function to report training stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_training` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_training(
                step, num_steps, learning_rate, report_stats,
                multigpu=self.n_gpu > 1)

    def _report_step(self, learning_rate, step, train_stats=None,
                     valid_stats=None):
        """
        Simple function to report stats (if report_manager is set)
        see `onmt.utils.ReportManagerBase.report_step` for doc
        """
        if self.report_manager is not None:
            return self.report_manager.report_step(
                learning_rate, step, train_stats=train_stats,
                valid_stats=valid_stats)

    def _maybe_save(self, step):
        """
        Save the model if a model saver is set
        """
        if self.model_saver is not None:
            self.model_saver.maybe_save(step)

            
def summary(args, b_list, device_id, pt, step, model):

    device = "cpu" if args.visible_gpus == '-1' else "cuda"
    if (pt != ''):
        test_from = pt
    else:
        test_from = args.test_from
    
    opt = vars(checkpoint['opt'])
    for k in opt.keys():
        if (k in model_flags):
            setattr(args, k, opt[k])
    #print(args)

    model.load_cp(checkpoint)
    model.eval()

    test_iter =data_loader.Dataloader(args, _lazy_dataset_loader(b_list),
                                  args.batch_size, device,
                                  shuffle=False, is_test=True)
    trainer = build_trainer(args, device_id, model, None)
    result = trainer.summary(test_iter,step)
    return result

def _tally_parameters(model):
    n_params = sum([p.nelement() for p in model.parameters()])
    return n_params

args.gpu_ranks = [int(i) for i in args.gpu_ranks.split(',')]
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

init_logger(args.log_file)
device = "cpu" if args.visible_gpus == '-1' else "cuda"
device_id = 0 if device == "cuda" else -1
model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers','encoder','ff_actv', 'use_interval','rnn_size']

In [59]:
import argparse
import json
import os
import time
import urllib3
from glob import glob
import collections
import six
import gc
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.utils import download as _download

def do_lang ( openapi_key, text ) :
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU"
    requestJson = {  
        "argument": {
            "text": text,
            "analysis_code": "morp"
        }
    }
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  openapi_key},
        body=json.dumps(requestJson)
    )

    json_data = json.loads(response.data.decode('utf-8'))
    json_result = json_data["result"]
    
    if json_result == -1:
        json_reason = json_data["reason"]
        if "Invalid Access Key" in json_reason:
            logger.info(json_reason)
            logger.info("Please check the openapi access key.")
            sys.exit()
        return "openapi error - " + json_reason
    else:
        json_data = json.loads(response.data.decode('utf-8'))
    
        json_return_obj = json_data["return_object"]
        
        return_result = ""
        json_sentence = json_return_obj["sentence"]
        for json_morp in json_sentence:
            for morp in json_morp["morp"]:
                return_result = return_result+str(morp["lemma"])+"/"+str(morp["type"])+" "

        return return_result

def get_kobert_vocab(cachedir="./tmp/"):
    # Add BOS,EOS vocab
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/tokenizers/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        #"fname": "/opt/ml/SumAI/bertsum/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece",
        "chksum": "ae5711deb3",
    }
    
    vocab_info = tokenizer
    vocab_file = _download(
        #vocab_info["url"], vocab_info["fname"], vocab_info["chksum"], cachedir=cachedir
        vocab_info["url"], vocab_info["chksum"], cachedir=cachedir
    )

    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
        vocab_file[0], padding_token="[PAD]", bos_token="[BOS]", eos_token="[EOS]"
    )
    return vocab_b_obj

    
class BertData():
    def __init__(self, vocab):
        self.tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)
        #self.tokenizer = Tokenizer(vocab_file_path)
        self.sep_vid = self.tokenizer.vocab['[SEP]']
        self.cls_vid = self.tokenizer.vocab['[CLS]']
        self.pad_vid = self.tokenizer.vocab['[PAD]']

    def preprocess(self, src):

        if (len(src) == 0):
            return None

        original_src_txt = [''.join(s) for s in src]


        idxs = [i for i, s in enumerate(src) if (len(s) > 0)]

        src = [src[i][:20000] for i in idxs]
        src = src[:10000]

        if (len(src) < 3):
            return None

        src_txt = [''.join(sent) for sent in src]
        text = ' [SEP] [CLS] '.join(src_txt)
        src_subtokens = text.split(' ')
        src_subtokens = src_subtokens[:510]
        src_subtokens = ['[CLS]'] + src_subtokens + ['[SEP]']

        src_subtoken_idxs = self.tokenizer.convert_tokens_to_ids(src_subtokens)
        _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if (i % 2 == 0):
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]
        cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == self.cls_vid]
        labels = None
        tgt_txt = None
        src_txt = [original_src_txt[i] for i in idxs]
        return src_subtoken_idxs, labels, segments_ids, cls_ids, src_txt, tgt_txt
    
def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")
        
class Tokenizer(object):
    
    def __init__(self, vocab_file_path):
        self.vocab_file_path = vocab_file_path
        """Loads a vocabulary file into a dictionary."""
        vocab = collections.OrderedDict()
        index = 0
        with open(self.vocab_file_path, "r", encoding='utf-8') as reader:

            while True:
                token = convert_to_unicode(reader.readline())
                if not token:
                    break

          ### joonho.lim @ 2019-03-15
                if token.find('n_iters=') == 0 or token.find('max_length=') == 0 :

                    continue
                token = token.split('\t')[0].strip('_')

                token = token.strip()
                vocab[token] = index
                index += 1
        self.vocab = vocab
        
    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            try:
                ids.append(self.vocab[token])
            except:
                ids.append(1)
        if len(ids) > 10000:
            raise ValueError(
                "Token indices sequence length is longer than the specified maximum "
                " sequence length for this BERT model ({} > {}). Running this"
                " sequence through BERT will result in indexing errors".format(len(ids), 10000)
            )
        return ids


def _lazy_dataset_loader(pt_file):
    dataset = pt_file    
    yield dataset
    
def News_to_input(text, openapi_key):
    newstemp = do_lang(openapi_key, text)
    news = newstemp.split(' ./SF ')[:-1]
    vocab = get_kobert_vocab()
    tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)
    bertdata = BertData(vocab)
    sent_labels = [0] * len(news)
    tmp = bertdata.preprocess(news)
    #print(tmp)
    b_data_dict = {"src":tmp[0],
               "tgt": [0],
               "labels":[0,0,0],
               "src_sent_labels":sent_labels,
               "segs":tmp[2],
               "clss":tmp[3],
               "src_txt":tmp[4],
               "tgt_txt":'hehe'}
    b_list = []
    b_list.append(b_data_dict) 
    return b_list

## 추출요약 테스트

In [60]:
openapi_key = '9318dc23-24ac-4b59-a99e-a29ec170bf02'
chat_history = "뉴스 "
user_input = '''
일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다.
정부 대변인인 가토 가쓰노부(加藤勝信) 관방장관은 이날 오전 정례 기자회견에서 관련 질문에 “말씀하신 보도와 같은 사실이 없는 것으로 안다”고 밝혔다.
앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다.
한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다.
가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다.
그는 한국 측의 독도방어훈련에 ‘어떤 대항 조치를 생각하고 있느냐’는 질문에는 “한국 해군의 훈련에 대해 정부로서는 강한 관심을 가지고 주시하는 상황이어서 지금 시점에선 논평을 삼가겠다”고 말을 아꼈다.
가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다.
그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다.
'''


bot_input_ids = News_to_input(chat_history + user_input, openapi_key)
print('------------------------------------------------------------')


#logger.info('Loading checkpoint from %s' % test_from)
checkpoint = torch.load('../models/bert_classifier/model_step_1000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
    
chat_history_ids = summary(args, bot_input_ids, -1, '', None, model)
print(chat_history_ids)
print('------------------------------------------------------------')
pred_lst = list(chat_history_ids[0][:3])
print(pred_lst)
print('------------------------------------------------------------')
final_text = ''
for i,a in enumerate(user_input.split('.')):
    if i in pred_lst:
        print(i, a)
        final_text = final_text+a+'. '
chat_history = user_input + '<token>' +final_text
print('------------------------------------------------------------')
print(final_text)
print('------------------------------------------------------------')

using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
------------------------------------------------------------


[2023-01-17 15:50:36,675 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-17 15:50:36,676 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



[[0 7 3 1 4 5 2 6]]
------------------------------------------------------------
[0, 7, 3]
------------------------------------------------------------
0 
일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다
3 
한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다
7 
그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다
------------------------------------------------------------

일본 정부가 문재인 대통령이 도쿄올림픽을 계기로 일본을 방문하는 방향으로 한일 양국이 조율하고 있다는 15일 요미우리신문의 보도를 부인했다. 
한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다. 
그러면서 “다케시마 문제에 대해서는 계속 우리나라의 영토, 영해, 영공을 단호히 지키겠다는 결의로 냉정하고 의연하게 대응해갈 생각”이라고 밝혔다. 
------------------------------------------------------------


In [80]:
checkpoint = torch.load('../models/bert_classifier2/model_step_35000.pt', map_location=lambda storage, loc: storage)
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device, load_pretrained_bert=False, bert_config = config)
    
def get_topk_sentences(k, user_input, model):
    bot_input_ids = News_to_input(user_input, openapi_key)
    
    chat_history_ids = summary(args, bot_input_ids, -1, '', None, model)
    pred_lst = list(chat_history_ids[0][:k])
    final_text = []
    for i,a in enumerate(user_input.split('.')):
        if i in pred_lst:
            #print(i, a)
            final_text.append((i, a+'. '))
    #print('------------------------------------------------------------')
    return final_text
    
    
print(get_topk_sentences(4, user_input, model))

[2023-01-17 15:59:07,136 INFO] loading archive file ../001_bert_morp_pytorch
[2023-01-17 15:59:07,138 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 30349
}



using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
[(2, '\n앞서 요미우리는 한국 측이 도쿄올림픽을 계기로 한 문 대통령의 방일을 타진했고, 일본 측은 수용하는 방향이라고 이날 보도했다. '), (3, '\n한국 측은 문 대통령의 방일 때 스가 요시히데(菅義偉) 총리와 처음으로 정상회담을 하겠다는 생각이라고 요미우리는 전했다. '), (4, '\n가토 장관은 한일 정상회담에 대한 일본 정부의 자세에 대해 “그런 사실이 없기 때문에 가정의 질문에 대해 답하는 것을 삼가겠다”고 말했다. '), (6, '\n가토 장관은 “다케시마(竹島·일본이 주장하는 독도의 명칭)는 역사적 사실에 비춰봐도, 국제법상으로도 명백한 일본 고유의 영토”라며 독도 영유권 주장을 되풀이했다. ')]


## Preprocess

In [81]:
# 공백 정리
import re, unicodedata
from string import whitespace, punctuation
from pykospacing import Spacing
from nltk import word_tokenize, sent_tokenize
import pandas as pd

def clean_byline(text):
    # byline
    pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
    pattern_others = re.compile(r'\.([^\.]*(?:기자|특파원|교수|작가|대표|논설|고문|주필|부문장|팀장|장관|원장|연구원|이사장|위원|실장|차장|부장|에세이|화백|사설|소장|단장|과장|기획자|큐레이터|저작권|평론가|©|©|ⓒ|\@|\/|=|▶|무단|전재|재배포|금지|\[|\]|\(\))[^\.]*)$')
    result = pattern_email.sub('', text)
    result = pattern_url.sub('', result)

    # 본문 시작 전 꺽쇠로 쌓인 바이라인 제거
    pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
    result = pattern_bracket.sub('', result).strip()
    return result


def text_filter(text):
    text = clean_byline(text)
    exclude_pattern = re.compile(r'[^\% 0-9a-zA-Zㄱ-ㅣ가-힣.,]+')
    exclusions = exclude_pattern.findall(text)
    result = exclude_pattern.sub(' ', text).strip()
    spacing = Spacing()
    kospacing_txt = spacing(result) 
    sentences = sent_tokenize(kospacing_txt)
    return sentences

def make_df(name, model):
    start = time.time()
    df = pd.read_csv(f'../{name}.csv')
    text = []
    top3 = []
    for context in df['context']:
        preprocess = text_filter(context)
        text.append([(i,v) for i,v in enumerate(preprocess)])
        top3.append(get_topk_sentences(len(preprocess)//4+1, ' '.join(preprocess), model))
    df['text'] = text
    df['topk'] = top3
    print(f"{time.time()-start:.4f} sec")
    return df


In [82]:
text_filter("삼성전자가 갤럭시 버즈2 프로                소프트웨어 업데이트를 통해 스마트폰 카메라 사용성을 한 단계 더 높였습니다. 삼성전자는 휴대전화로 동영상을 찍을 때 360도 사운드 녹음이 가능하도록 갤럭시 버즈2 프로와 갤럭시Z 플립4, 폴드4의 소프트웨어 업데이트를 시행한다고 밝혔습니다. 소프트웨어 업데이트를 통해 버즈2 프로 좌우에 있는 마이크로 사용자가 듣는 그대로의 소리를 생생하게 녹음할 수 있다고 삼성전자 측은 설명했습니다. ※ '당신의 제보가 뉴스가 됩니다' [카카오톡] YTN 검색해 채널 추가 [전화] 02-398-8585 [메일]")

1/1 [==============================] - 0s 31ms/step


['삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라 사용성을 한 단계 더 높였습니다.',
 '삼성전자는 휴대전화로 동영상을 찍을 때 360도 사운드 녹음이 가능하도록 갤럭시 버즈 2 프로와 갤럭시Z 플립4, 폴드 4의 소프트웨어 업데이트를 시행한다고 밝혔습니다.',
 '소프트웨어 업데이트를 통해 버즈 2 프로 좌우에 있는 마 이 크로 사용자가 듣는 그대로의 소리를 생생하게 녹음할 수 있다고 삼성전자 측은 설명했습니다.',
 '당신의 제보가 뉴스가 됩니다 카카오톡 YTN 검색해 채널 추가 전화 02 398 8585 메일']

In [83]:
df = make_df('result5', model)
df

1/1 [==============================] - 0s 30ms/step
using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
1/1 [==============================] - 0s 29ms/step
using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
1/1 [==============================] - 0s 118ms/step
using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


,Unnamed: 0,title,originallink,link,description,pubDate,context,text,topk
0,0,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…&quot...",https://www.yna.co.kr/view/AKR2023011303230001...,https://n.news.naver.com/mnews/article/001/001...,<b>삼성전자</b>는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할...,"Fri, 13 Jan 2023 09:12:00 +0900",= 삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도...,"[(0, 삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수...","[(2, 녹음에는 차세대 블루투스 오디오 기술 표준인 LE 오디오가 적용됐다. ..."
1,1,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…스마트폰 ...",http://www.ezyeconomy.com/news/articleView.htm...,http://www.ezyeconomy.com/news/articleView.htm...,<b>삼성전자</b>는 갤럭시 <b>버즈2 프로</b>와 갤럭시 Z 플립4·폴드4의...,"Mon, 16 Jan 2023 10:04:00 +0900",[=김진이 기자] 삼성전자는 갤럭시 버즈2 프로와 갤럭시 Z 플립4·폴드4의 소프트...,"[(0, 삼성전자는 갤럭시 버즈 2 프로와 갤럭시 Z 플립4 폴드4의 소프트웨어 S...","[(5, 삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤..."
2,2,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…&quot...",https://www.joongang.co.kr/article/25133414,https://n.news.naver.com/mnews/article/025/000...,"<b>삼성전자</b>에 따르면, 이날부터 소프트웨어를 <b>업데이트</b>하면 <b...","Fri, 13 Jan 2023 10:46:00 +0900",삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 ...,"[(0, 삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수...","[(2, 녹음에는 차세대 블 루투스 오디오 기술 표준인 LE 오디오가 적용됐다. ..."
3,3,<b>삼성전자 버즈2 프로 업데이트</b>···“360도 소리 녹음 지원”,https://www.khan.co.kr/economy/business/articl...,https://n.news.naver.com/mnews/article/032/000...,<b>삼성전자</b>는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음...,"Fri, 13 Jan 2023 11:58:00 +0900",삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 수 있도록...,"[(0, 삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 ...","[(2, 녹음에는 차세대 블루투스 오디오 기술 표 준인 LE 오디오가 적용됐다. ..."
4,4,"[기업] <b>삼성</b>, 무선이어폰 <b>업데이트</b>로 360도 사운드 녹음 가능",https://www.ytn.co.kr/_ln/0102_202301131647168253,https://n.news.naver.com/mnews/article/052/000...,<b>삼성전자</b>가 갤럭시 <b>버즈2 프로</b> 소프트웨어 <b>업데이트</...,"Fri, 13 Jan 2023 16:47:00 +0900",삼성전자가 갤럭시 버즈2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라 사용성을 ...,"[(0, 삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라...","[(0, 삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라..."


In [88]:
i = 2
df['text'][i]

[(0,
  '삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈 2 프로 및 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 13일부터 시행한다.'),
 (1,
  '삼성전자에 따르면, 이 날부터 소프트웨어를 업데이트하면 버즈 2 프로 좌우에 있는 마이크가 모두 사용돼 360도에서 들려오는 소리를 플립 4 폴드4로 녹음할 수 있다.'),
 (2,
  '녹음에는 차세대 블 루투스 오디오 기술 표준인 LE 오디오가 적용됐다.이와 함께 삼성전자는 다음 달 갤럭시 워치5와 워치 4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 전했다.'),
 (3,
  '삼성전자는 갤럭시 제품 간 연동을 통해 더욱 향상된 카메라 경험을 제공할 것 이라며 다음 달 진행되는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많은 내용을 소개할 것 이라고 말했 다.')]

In [89]:
df['topk'][i]

[(2, ' 녹음에는 차세대 블 루투스 오디오 기술 표준인 LE 오디오가 적용됐다. '),
 (4,
  ' 삼성전자는 갤럭시 제품 간 연동을 통해 더욱 향상된 카메라 경험을 제공할 것 이라며 다음 달 진행되는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많은 내용을 소개할 것 이라고 말했 다. ')]

In [71]:
df.to_csv('../result5_pre.csv')

In [90]:
df = make_df('result10', model)
df.to_csv('../result10_pre.csv')

1/1 [==============================] - 0s 28ms/step
using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
1/1 [==============================] - 0s 28ms/step
using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
1/1 [==============================] - 0s 26ms/step
using cached model. /opt/ml/KorBertSum/src/./tmp/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /opt/ml/KorBertSum/src/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
1

In [91]:
df

,Unnamed: 0,title,originallink,link,description,pubDate,context,text,topk
0,0,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…&quot...",https://www.yna.co.kr/view/AKR2023011303230001...,https://n.news.naver.com/mnews/article/001/001...,<b>삼성전자</b>는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할...,"Fri, 13 Jan 2023 09:12:00 +0900",= 삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도...,"[(0, 삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수...","[(2, 녹음에는 차세대 블루투스 오디오 기술 표준인 LE 오디오가 적용됐다. ..."
1,1,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…스마트폰 ...",http://www.ezyeconomy.com/news/articleView.htm...,http://www.ezyeconomy.com/news/articleView.htm...,<b>삼성전자</b>는 갤럭시 <b>버즈2 프로</b>와 갤럭시 Z 플립4·폴드4의...,"Mon, 16 Jan 2023 10:04:00 +0900",[=김진이 기자] 삼성전자는 갤럭시 버즈2 프로와 갤럭시 Z 플립4·폴드4의 소프트...,"[(0, 삼성전자는 갤럭시 버즈 2 프로와 갤럭시 Z 플립4 폴드4의 소프트웨어 S...","[(5, 삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤..."
2,2,"<b>삼성전자</b>, <b>버즈2프로</b> SW <b>업데이트</b>…&quot...",https://www.joongang.co.kr/article/25133414,https://n.news.naver.com/mnews/article/025/000...,"<b>삼성전자</b>에 따르면, 이날부터 소프트웨어를 <b>업데이트</b>하면 <b...","Fri, 13 Jan 2023 10:46:00 +0900",삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 ...,"[(0, 삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수...","[(2, 녹음에는 차세대 블 루투스 오디오 기술 표준인 LE 오디오가 적용됐다. ..."
3,3,<b>삼성전자 버즈2 프로 업데이트</b>···“360도 소리 녹음 지원”,https://www.khan.co.kr/economy/business/articl...,https://n.news.naver.com/mnews/article/032/000...,<b>삼성전자</b>는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음...,"Fri, 13 Jan 2023 11:58:00 +0900",삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 수 있도록...,"[(0, 삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 ...","[(2, 녹음에는 차세대 블루투스 오디오 기술 표 준인 LE 오디오가 적용됐다. ..."
4,4,"[기업] <b>삼성</b>, 무선이어폰 <b>업데이트</b>로 360도 사운드 녹음 가능",https://www.ytn.co.kr/_ln/0102_202301131647168253,https://n.news.naver.com/mnews/article/052/000...,<b>삼성전자</b>가 갤럭시 <b>버즈2 프로</b> 소프트웨어 <b>업데이트</...,"Fri, 13 Jan 2023 16:47:00 +0900",삼성전자가 갤럭시 버즈2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라 사용성을 ...,"[(0, 삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라...","[(0, 삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라..."
5,5,"갤워치로 스마트폰 카메라 줌 인·아웃…<b>삼성전자</b>, 갤Z플립4 등 기능 <...",https://sports.donga.com/article/all/20230116/...,https://n.news.naver.com/mnews/article/382/000...,<b>삼성전자</b>는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 ...,"Mon, 16 Jan 2023 10:01:00 +0900",삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 '갤럭시버즈2...,"[(0, 삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭...","[(3, 갤럭시 버즈 2 프로의 좌우에 위치한 마이크를 모두 사용해 360도 사운..."
6,6,"<b>삼성전자</b>, <b>버즈2프로</b>·갤워치 SW <b>업데이트</b>…&...",http://news.tf.co.kr/read/economy/1992273.htm,https://n.news.naver.com/mnews/article/629/000...,이번 <b>업데이트</b>는 이날부터 시행되며 스마트폰으로 동영상 촬영 시 360 ...,"Fri, 13 Jan 2023 10:57:00 +0900",삼성전자가 스마트폰과 웨어러블 기기의 연동을 강화해 카메라 사용성을 개선한다.삼성전...,"[(0, 삼성전자가 스마트폰과 웨어러블 기기의 연동을 강화해 카메라 사용성을 개선한...","[(4, 360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오 를..."
7,7,"<b>삼성전자</b>, <b>버즈2 프로</b> SW<b>업데이트</b>…360 오...",http://www.m-i.kr/news/articleView.html?idxno=...,http://www.m-i.kr/news/articleView.html?idxno=...,사진=<b>삼성전자</b> 제공 <b>삼성전자</b>는 갤럭시 <b>버즈2 프로<...,"Fri, 13 Jan 2023 09:04:00 +0900",갤럭시 워치5·워치4 시리즈에 카메라 줌인·줌아웃 기능 탑재 사진=삼성전자 제공 [...,"[(0, 갤럭시 워치5 워치4 시리즈에 카메라 줌인 줌 아웃 기능 탑재 사진 삼성전...","[(4, LE 오디오 기술은 보다 풍부하고 섬세한 사운드, 매끄러운 연결성, 향상..."
8,8,[<b>삼성</b> 신성장동력] 新모델 없는 &apos;갤럭시 <b>버즈</b>&a...,https://dealsite.co.kr/articles/97403,https://dealsite.co.kr/articles/97403,갤럭시 <b>버즈2 프로</b>와 갤럭시 Z 플립4·폴드4의 소프트웨어 <b>업데이...,"Mon, 16 Jan 2023 08:36:00 +0900",약 6개월 사이로 신제품을 출시하던 삼성전자는 이후 1년 만인 2022년 8월 버즈...,"[(0, 약 6개월 사이로 신제품을 출시하던 삼성전자는 이후 1년 만인 2022년 ...","[(2, 13일 업계에 따르면 삼성전자는 2020년 2월 버즈 플러스 모델명 R1..."
9,9,"<b>삼성전자</b>, <b>버즈2 프로</b> SW<b>업데이트</b>… 360도...",https://www.viva100.com/main/view.php?key=2023...,https://www.viva100.com/main/view.php?key=2023...,"<b>삼성전자</b>, <b>버즈2 프로</b> SW<b>업데이트</b>… 360도...","Fri, 13 Jan 2023 09:32:00 +0900",삼성전자는 갤럭시버즈2 프로와 갤럭시Z 플립4·폴드4의 소프트웨어 업데이트를 통해 ...,"[(0, 삼성전자는 갤럭시 버즈 2 프로와 갤럭시Z 플립4 폴드4의 소프트웨어 업데...","[(4, 삼성전자는 갤럭시 버즈 2 프로와 갤럭시Z플립4와 폴드 4의 소프트웨어 ..."


In [92]:
df['text'][6]

[(0, '삼성전자가 스마트폰과 웨어러블 기기의 연동을 강화해 카메라 사용성을 개선한다.'),
 (1,
  '삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 무선어폰 갤럭시 버즈 2 프로 와 스마트폰 갤럭시Z플립 4 폴드4의 소프트웨어 업데이트를 시행한다고 13일 밝혔다.'),
 (2, '이번 업데이트는 이 날부터 시행되며 스마트폰으로 동영상 촬영 시 360 오디오 레코딩 기능을 지원 하는 것이 특징이다.'),
 (3,
  '삼성전자는 갤럭시 버즈 2 프로의 좌우에 위치한 마이크를 모두 사용해 360도 사운드를 포착한 다 며 이를 통해 전문 장비 없이도 언제 어디서든, 누구나 고품질의 실감 나는 오디오를 동영상에 담을 수 있다 고 설명했다.360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오 를 적용했다.'),
 (4, 'LE 오디오 기술은 풍부하고  섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공하는 것이 특징이다.'),
 (5,
  '아울러 삼성전자는 스마트워치 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔다.'),
 (6, '해당 기능은 2월 중 기존의 워치 카메라 컨트롤러 앱에 추가될 예정이다.'),
 (7,
  '사용자는 시계 화면을 손가락으로 늘 리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 된다.'),
 (8, '이를 통해 스마트폰을 먼 거리에 세워 두고 촬영할 때 손쉽게 화면 배율 조정을 할 수 있다.'),
 (9, '해당 기능은 원 UI 소프트웨어 지원 스마트폰과 연동해 이용할 수 있다.'),
 (10, '적용 모델과 세부 시점은 추후 안내된다.'),
 (11,
  '삼성전자 관계자는 삼성전자는 갤럭시  제품 간 연동을 통해 보다 향상된 카메라 경험을 제공할 것 이라며 다음 달 1일 열리는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많

In [93]:
df['topk'][6]

[(4, '360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오 를 적용했다. '),
 (5, ' LE 오디오 기술은 풍부하고  섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공하는 것이 특징이다. '),
 (6,
  ' 아울러 삼성전자는 스마트워치 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔다. '),
 (8,
  ' 사용자는 시계 화면을 손가락으로 늘 리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 된다. ')]

In [99]:
eval(str(df['topk'][6]))[:1]

[(4, '360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오 를 적용했다. ')]

## K-means 클러스터링

In [147]:
first_sentences = []
preprocessed_documents = []
for a,b in zip(df['text'], df['topk']):
    if(eval(str(b))[0][0] == 0):
        first_sentences += eval(str(b))[:1]
        preprocessed_documents += eval(str(b))[1:]
    else:
        first_sentences += eval(str(a))[:1]
        preprocessed_documents += eval(str(b))
    
print(len(first_sentences))
print(len(preprocessed_documents))

10
38


In [110]:
str(preprocessed_documents[0][0])

"(2, ' 녹음에는 차세대 블루투스 오디오 기술  표준인 LE 오디오가 적용됐다. ')"

In [111]:
document_df = pd.DataFrame()
document_df['opinion_text'] = [str(t) for t in preprocessed_documents]
len(document_df)

38

In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string
from sklearn.cluster import KMeans


# 토큰화한 각 단어들의 원형들을 리스트로 담아서 반환
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 텍스트를 Input으로 넣어서 토큰화시키고 토큰화된 단어들의 원형들을 리스트로 담아 반환
def LemNormalize(text):
    # .translate인자에 구두점 dict넣어주어서 구두점 삭제해준 상태로 토큰화시키기!
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

def get_clustered_df(sentences, clusternum):
    print(clusternum)
    document_df = pd.DataFrame()
    document_df['opinion_text'] = [str(t) for t in sentences]
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

    lemmar = WordNetLemmatizer()
    tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize,
                            ngram_range=(1,2),
                            min_df=0.05, max_df=0.85)

    # fit_transform으로 위에서 구축한 도구로 텍스트 벡터화
    ftr_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
    # K-means로 3개 군집으로 문서 군집화시키기

    kmeans = KMeans(n_clusters=clusternum, max_iter=10000, random_state=42)
    # 비지도 학습이니 feature로만 학습시키고 예측
    cluster_label = kmeans.fit_predict(ftr_vect)
    
    # 군집화한 레이블값들을 document_df 에 추가하기
    document_df['cluster_label'] = cluster_label
    return document_df.sort_values(by=['cluster_label'])
    

In [180]:
clusternum = len(preprocessed_documents)//7
document_df = get_clustered_df(preprocessed_documents, clusternum)
print(document_df)

5
                                         opinion_text  cluster_label
18  (8, ' 사용자는 시계 화면을 손가락으로 늘 리거나 줄이는 핀치 동작이나, 시계의...              0
31  (15, ' 하지만 이번에는 소프트웨어 업데이트 이외에  새로운 신제품 출시는 이뤄...              0
28  (7, ' 신제품 대신 업데이트로 대응 업계에서는 올해 삼성전자의 무선 이어폰 신제...              0
27  (6, ' 딜 사이트 김민기 기자 삼성전자의 새로운 먹거리로 보였던  무선 이어폰이...              0
26  (5, ' 하지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 ...              0
22               (10, ' 적용 모델과 세부 시점은 추후 안내될 예정이다. ')              0
20     (6, ' 이 기능은 다음 달 중 기존의 워치 카메라 컨트롤러 앱에 추가된다. ')              0
36  (15, ' 해당 기능은 원 UI 소프트웨어 지원 스마트폰과 연동해 이 용할 수 있...              0
17  (6, ' 아울러 삼성전자는 스마트워치 갤럭시 워치5와 워치 4 시리즈도 소프트웨어...              0
13  (4, ' 갤럭시 워치5와 워치 4 시리즈의 카메라 줌 인 줌 아웃 기능은 2월 중...              0
37                   (16, ' 적용 모델과 세부 시점은 추후 안내된다. ')              0
8   (4, ' 삼성전자는 갤럭시 제품 간 연동을 통해 더욱 향상된 카메라 경험을 제공할...              0
1   (3, '이와 함께 삼성전자는 다음 달 갤럭시 워치5와 워치 4 시리즈에 카메라 줌...              0
5   (14, ' 사용자는 시계 화면을 손가락으로 늘리거

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [181]:
def print_clustered_df(df, clabel):
    for i,t in enumerate(df[df['cluster_label'] == clabel]['opinion_text']):
        print(i, t)

In [183]:
print_clustered_df(document_df, 0)

0 (8, ' 사용자는 시계 화면을 손가락으로 늘 리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 된다. ')
1 (15, ' 하지만 이번에는 소프트웨어 업데이트 이외에  새로운 신제품 출시는 이뤄지지 않는다. ')
2 (7, ' 신제품 대신 업데이트로 대응 업계에서는 올해 삼성전자의 무선 이어폰 신제품 출시는 없을 것으로 전망하고 있다. ')
3 (6, ' 딜 사이트 김민기 기자 삼성전자의 새로운 먹거리로 보였던  무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있다. ')
4 (5, ' 하지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이다. ')
5 (10, ' 적용 모델과 세부 시점은 추후 안내될 예정이다. ')
6 (6, ' 이 기능은 다음 달 중 기존의 워치 카메라 컨트롤러 앱에 추가된다. ')
7 (15, ' 해당 기능은 원 UI 소프트웨어 지원 스마트폰과 연동해 이 용할 수 있다. ')
8 (6, ' 아울러 삼성전자는 스마트워치 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔다. ')
9 (4, ' 갤럭시 워치5와 워치 4 시리즈의 카메라 줌 인 줌 아웃 기능은 2월 중 기존 워치 카메라 컨트롤러 앱에 추가된다 . ')
10 (16, ' 적용 모델과 세부 시점은 추후 안내된다. ')
11 (4, ' 삼성전자는 갤럭시 제품 간 연동을 통해 더욱 향상된 카메라 경험을 제공할 것 이라며 다음 달 진행되는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많은 내용을 소개할 것 이라고 말했 다. ')
12 (3, '이와 함께 삼성전자는 다음 달 갤럭시 워치5와 워치 4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 전했다. ')
13 (14, ' 사용자는 시계 화면을 손가락으로 늘리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤

In [184]:
print_clustered_df(document_df, 1)

0 (2, ' 녹음에는 차세대 블루투스 오디오 기술  표준인 LE 오디오가 적용됐다. ')
1 (8, ' 특히 360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오 디오를 적용했다. ')
2 (2, ' 녹음에는 차세대 블루투스 오디오 기술 표 준인 LE 오디오가 적용됐다. ')
3 (4, '360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오 를 적용했다. ')
4 (5, ' LE 오디오 기술은 풍부하고  섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공하는 것이 특징이다. ')
5 (2, ' 녹음에는 차세대 블 루투스 오디오 기술 표준인 LE 오디오가 적용됐다. ')
6 (4, ' LE 오디오 기술은 보다 풍부하고 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공한다고 회사는 설명했다. ')


In [199]:
summarize_topic2(document_df, 0)

1.6903 sec
삼성전자의 새로운 먹거리로 보였던 무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이며  운영체제 업데이트 이외에  새로운 신제품 출시는 이뤄지지 않는다.
6 1
-----------------------------------------------
2.0762 sec
삼성전자는 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔으며 갤럭시 워치5와 워치 4 시리즈의 카메라 줌 인 줌 아웃 기능은 2월 중 기존 워치 카메라 컨트롤러 앱에 추가된다.
6 2
-----------------------------------------------
1.4541 sec
삼성전자는 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있는 소프트웨어 업데이트 이외에  새로운 신제품 출시는 이뤄지지 않으며 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이다.       
6 3
-----------------------------------------------
2.0094 sec
삼성전자는 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔으며 갤럭시 워치5와 워치 4 시리즈의 카메라 줌 인 줌 아웃 기능은 2월 중 기존 워치 카메라 컨트롤러 앱에 추가된다.
6 4
-----------------------------------------------
1.7800 sec
갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있는 소프트웨어 업데이트 이외에  새로운 신제품 출시는 이뤄지지 않으며 삼성전자는 무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출

'삼성전자의 새로운 먹거리로 보였던 무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이며 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔으며 갤럭시 워치5와 워치 4 시리즈의 카메라 줌 인 줌 아웃 기능은 2월 중 기존 워치 카메라 컨트롤러 앱에 추가된다.'

In [230]:
sum_result = []
for c in range(5):
    print_clustered_df(document_df, c)
    print('---------------------------------------------------')
    summ = summarize_topic2(document_df, c)
    sum_result.append(summ)
    print(summ)
    print('***************************************************')
    

0 (8, ' 사용자는 시계 화면을 손가락으로 늘 리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 된다. ')
1 (15, ' 하지만 이번에는 소프트웨어 업데이트 이외에  새로운 신제품 출시는 이뤄지지 않는다. ')
2 (7, ' 신제품 대신 업데이트로 대응 업계에서는 올해 삼성전자의 무선 이어폰 신제품 출시는 없을 것으로 전망하고 있다. ')
3 (6, ' 딜 사이트 김민기 기자 삼성전자의 새로운 먹거리로 보였던  무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있다. ')
4 (5, ' 하지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이다. ')
5 (10, ' 적용 모델과 세부 시점은 추후 안내될 예정이다. ')
6 (6, ' 이 기능은 다음 달 중 기존의 워치 카메라 컨트롤러 앱에 추가된다. ')
7 (15, ' 해당 기능은 원 UI 소프트웨어 지원 스마트폰과 연동해 이 용할 수 있다. ')
8 (6, ' 아울러 삼성전자는 스마트워치 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔다. ')
9 (4, ' 갤럭시 워치5와 워치 4 시리즈의 카메라 줌 인 줌 아웃 기능은 2월 중 기존 워치 카메라 컨트롤러 앱에 추가된다 . ')
10 (16, ' 적용 모델과 세부 시점은 추후 안내된다. ')
11 (4, ' 삼성전자는 갤럭시 제품 간 연동을 통해 더욱 향상된 카메라 경험을 제공할 것 이라며 다음 달 진행되는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많은 내용을 소개할 것 이라고 말했 다. ')
12 (3, '이와 함께 삼성전자는 다음 달 갤럭시 워치5와 워치 4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 전했다. ')
13 (14, ' 사용자는 시계 화면을 손가락으로 늘리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤

In [231]:
sum_result

['삼성전자는 무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이라고 밝혔으며 갤럭시 워치5와 워치 4 시리즈도 소프트웨어 업데이트를 통해 스마트폰 카메라를 원격으로 줌 인 줌 아웃 할 수 있는 기능을 추가한다고 밝혔으며 갤럭시 워치5와 워치 4 시리즈의 카메라 줌 인 줌 아웃 기능은 2월 중 기존 워치 카메라 컨트롤러 앱에 추가된다.',
 '360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오를 적용했으며 녹음에는 풍부하고 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공한다고 회사는 설명했다.',
 '삼성전자는 갤럭시 버즈 2 프로와 갤럭시Z플립4와 폴드4의 소프트웨어 업데이트를 통해 360도 오디오 레코딩 기능을 지원하며 촬영자는 연결된 버즈 2 프로를 통해 본인이 듣는 그대로의 생생한 몰입형 사운드를 녹음할 수 있다고 밝혔다.',
 '스마트폰을 먼 거리에 세워두고 단체 사진이나 단독 셀카를 찍을 때 카메라가 있는 곳까지 돌아갈 필요 없이 손목에서 바로 손쉽게 화면 배율 조정을 할 수 있게 되었으며, 편의를 통해 스마트폰을 먼 거리에 세워두고 단체 사진이나 단독 셀카를 찍을 때 카메라가 있는 곳까지 돌아갈 필요 없이 손목에서 바로 손쉽게 화면 배율 조정이 가능해진다.',
 '삼성전자는 2020년 2월 버즈 플러스 모델명 R175 를 첫 출 시한 이후 같은 해 8월 버 즈 라이브 R180 를 선 보였으며 2019년에 보급형 라인인 버즈 3가 신제품으로 출시되고 고급형인 버즈 프로3는 2025년에나 나올 것으로 예상하고 있다.']

In [240]:
clusternum = 1
first_df = get_clustered_df(first_sentences, clusternum)
print(first_df)

1
                                        opinion_text  cluster_label
0  (0, '삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수...              0
1  (0, '삼성전자는 갤럭시 버즈 2 프로와 갤럭시 Z 플립4 폴드4의 소프트웨어 S...              0
2  (0, '삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수...              0
3  (0, '삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 ...              0
4  (0, '삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라...              0
5  (0, '삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭...              0
6  (0, '삼성전자가 스마트폰과 웨어러블 기기의 연동을 강화해 카메라 사용성을 개선한...              0
7  (0, '갤럭시 워치5 워치4 시리즈에 카메라 줌인 줌 아웃 기능 탑재 사진 삼성전...              0
8  (0, '약 6개월 사이로 신제품을 출시하던 삼성전자는 이후 1년 만인 2022년 ...              0
9  (0, '삼성전자는 갤럭시 버즈 2 프로와 갤럭시Z 플립4 폴드4의 소프트웨어 업데...              0


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [244]:
sum_result2 = []
for c in range(clusternum):
    print_clustered_df(first_df, c)
    print('---------------------------------------------------')
    summ = summarize_topic(first_df, c)
    sum_result2.append(summ)
    print(summ)
    print('***************************************************')
    

0 (0, '삼성전자는 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈 2 프로 및 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 13일부터 시행한다고 밝혔다.')
1 (0, '삼성전자는 갤럭시 버즈 2 프로와 갤럭시 Z 플립4 폴드4의 소프트웨어 SW 업데이트를 13일부터 시행한다.')
2 (0, '삼성전자가 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈 2 프로 및 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 13일부터 시행한다.')
3 (0, '삼성전자는 스마트폰으로 동영상 촬영 시 전방위에서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈 2 프로 및 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 시행한다고 13일 밝혔다.')
4 (0, '삼성전자가 갤럭시 버즈 2 프로 소프트웨어 업데이트를 통해 스마트폰 카메라 사용성을 한 단계 더 높였습니다. ')
5 (0, '삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭시 버즈 2 프로 와 갤럭시Z플립 4 폴드 4 의 소프트웨어 업데이트를 단행했다.')
6 (0, '삼성전자가 스마트폰과 웨어러블 기기의 연동을 강화해 카메라 사용성을 개선한다.')
7 (0, '갤럭시 워치5 워치4 시리즈에 카메라 줌인 줌 아웃 기능 탑재 사진 삼성전자 제공 신지하 기자 삼성전자는 갤럭시 버즈 2 프로와 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 통해 360 오디오 레코딩 기능을 지원한다고 13일 밝혔다.')
8 (0, '약 6개월 사이로 신제품을 출시하던 삼성전자는 이후 1년 만인 2022년 8월 버즈 프로 2 R510 를 출시했다.')
9 (0, '삼성전자는 갤럭시 버즈 2 프로와 갤럭시Z 플립4 폴드4의 소프트웨어 업데이트를 통해 360 오디오 레코딩 기능을 지원한다.')
---------------------------------------------------
2.4916 sec
삼성전자는 스마트폰으로 스마트폰으로

In [245]:
sum_result2

['삼성전자는 스마트폰으로 스마트폰으로 동영상 촬영 시 전방위서 들려오는 소리를 녹음할 수 있도록 갤럭시 버즈 2 프로 및 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 13일부터 시행한다고 13일 밝혔다.  \n삼성전자는 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭시 버즈 2 프로 및 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 13일부터 시행한다고 밝혔다.']

## 생성 요약

In [136]:
import time
import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
import random

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

def make_summarization(sentences):
    text = '\n'.join(sentences)
    start = time.time()
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512, min_length=64,  eos_token_id=1)
    print(f"{time.time()-start:.4f} sec")
    sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    return sum_result

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [227]:
def make_summarization2(sentences):
    split = []
    for i in range(len(sentences)//8):
        split.append(sentences[:8])
        sentences = sentences[8:]

    for i in range(len(split)):
        if(len(sentences) == 0): break
        split[i].append(sentences.pop())
    
    if(len(sentences) != 0): split.append(sentences)
    
    split_sum = []
    for sentences in split:
        text = '\n'.join(sentences)
        start = time.time()
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512, min_length=48,  eos_token_id=1)
        print(f"{time.time()-start:.4f} sec")
        sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        print(sum_result)
        split_sum.append(sum_result)
        print(len(split), len(split_sum))
        print('-----------------------------------------------')
    
    if(len(split_sum)==1):
        return split_sum[0]
        
    text = '\n'.join(split_sum)
    start = time.time()
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

    summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512, min_length=48,  eos_token_id=1)
    print(f"{time.time()-start:.4f} sec")
    sum_result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
    return sum_result

In [228]:
def summarize_topic2(document_df, topic_num):
    sentences = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
    result = make_summarization2(sentences)
    return result

In [229]:
result = summarize_topic2(document_df, 1)
result

1.3985 sec
360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오를 적용했으며 녹음에는 풍부하고 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공한다고 회사는 설명했다.
1 1
-----------------------------------------------


'360 오디오 레코딩은 차세대 블루투스 오디오 기술 표준인 LE 오디오를 적용했으며 녹음에는 풍부하고 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공한다고 회사는 설명했다.'

In [137]:
sentences = []
for i,t in enumerate(document_df[document_df['cluster_label'] == 6]['opinion_text']):
    print(i, t)
    sentences.append(eval(t)[1])

0 (5, ' 하지만 올해는 수익성, 재고 문제 등으로 인해 이어폰 신제품을 출시하지 않을 예정이다. ')
1 (6, ' 딜 사이트 김민기 기자 삼성전자의 새로운 먹거리로 보였던  무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있다. ')
2 (7, ' 신제품 대신 업데이트로 대응 업계에서는 올해 삼성전자의 무선 이어폰 신제품 출시는 없을 것으로 전망하고 있다. ')
3 (15, ' 하지만 이번에는 소프트웨어 업데이트 이외에  새로운 신제품 출시는 이뤄지지 않는다. ')
4 (16, ' 삼성전자 관계자는 갤럭시 버즈 2 신제품 나온 지가 6개월 밖에 안 된 상황 이라며 올해 신제품 출시와 관련해서는 아직 잘 모르겠다 고 답했다. ')
5 (15, ' 해당 기능은 원 UI 소프트웨어 지원 스마트폰과 연동해 이 용할 수 있다. ')


In [138]:
result = make_summarization(sentences)
result

1.7708 sec


'삼성전자의 새로운 먹거리로 보였던 무선 이어폰이 애플이라는 큰 벽과 중국 기업의 저가 공세를 막지 못하면서 어려움을 겪고 있는 가운데, 삼성전자는 올해 무선 이어폰 신제품 출시는 없을 것으로 전망하고 업데이트로 대응 업계에서는 올해 무선 이어폰 신제품 출시는 없을 것으로 전망하고 있다.     '

In [140]:
def summarize_topic(document_df, topic_num):
    sentences = []
    for i,t in enumerate(document_df[document_df['cluster_label'] == topic_num]['opinion_text']):
        sentences.append(eval(t)[1])
    result = make_summarization(sentences)
    return result

In [139]:
result = summarize_topic(document_df, 0)
result

1.7528 sec


'더 풍부하고 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공하는 LE 오디오 기술은 보다 풍부하고 섬세한 사운드, 섬세한 사운드, 매끄러운 연결성, 향상된 배터리 수명을 제공하는 것이 특징이라고 회사는 설명했다.         '

In [141]:
result1 = summarize_topic(document_df, 1)
result1

2.8699 sec


'삼성전자는 갤럭시 스마트폰 동영상 촬영 시 360도 사운드 녹음이 가능하도록 갤럭시 버즈 2 프로와 갤럭시 Z 플립4 폴드4의 소프트웨어 업데이트를 시행하여 사용자가 직접 듣는 그대로의 생생한 사운드를 녹음할 수 있다고 밝혔다.  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

In [142]:
result2 = summarize_topic(document_df, 2)
result2

2.3841 sec


'삼성전자는 다음 워치5와 워치 4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 밝히며 다음 달 진행되는 갤럭시 언팩에서 새로운 갤럭시 스마트폰과 관련된 더 많은 내용을 소개할 것 이라고 말했다  \n 삼성전자는 다음 달 갤럭시 워치5와 워치 4 시리즈에 카메라 줌 인, 줌 아웃 기능을 추가한다고 전했다.'

In [143]:
result3 = summarize_topic(document_df, 3)
result3

2.5630 sec


'스마트폰을 먼 거리에 세워두고 단체 사진이나 단독 셀카를 찍을 때 카메라가 있는 곳까지 돌아갈 필요 없이 손목에서 바로 손쉽게 화면 배율 조정을 할 수 있게 되어 손목에서 바로 손쉽게 화면 배율 조정을 할 수 있게 되었으며,  \n 스마트폰을 먼 거리에 세워두고 단체 사진이나 단독 셀카를 찍을 때 카메라가 있는 곳까지 돌아갈 필요 없이 손목에서 바로 손쉽게 화면 배율 조정이 가능해진다.'

In [144]:
result4 = summarize_topic(document_df, 4)
result4

2.1447 sec


'360 오디오 레코딩은 차세대 블 루투스 오디오 기술 표 준인 LE 오디오가 적용됐다.  련련련련련련련련련련련련련련련련련련은 차세대 블루투스 오디오 기술 표 준인 LE 오디오를 적용했다.'

In [145]:
result5 = summarize_topic(document_df, 5)
result5

2.9359 sec


'전문 장비 없이도 누구나  언제 어디서든 고품질의 실감 나는 오디오를 동영상에 담을 수 있다는 게 삼성전자의 설명으로 사용자는 시계 화면을 손가락으로 늘리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 되었으며 사용자는 시계 화면을 손가락으로 늘 리거나 줄이는 핀치 동작이나, 시계의 베젤을 돌려 갤럭시 스마트폰의 카메라 줌을 원격으로 제어할 수 있게 되었다.'